# Indexy
- Index je neměnná sekvence určená k indexaci a řazení
- Může obsahovat duplicitní hodnoty
- Může obsahovat jakýkoliv datový typ
- Může mít více úrovní (jedná se o tzv. multiindex)



## Vybrané metody a atributy pro práci s indexem

### Atributy
- Index.values: Výpis hodnot indexu, viz https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.values.html
- Index.nlevels: Informace o počtu úrovní indexu, viz https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.nlevels.html
- Index.is_unique: Informace o tom, jestli jsou hodnoty indexu unikátní, viz https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.unique.html
- Index.hasnans: Informace o tom, jestli je v indexu hodnota NaN, viz https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.hasnans.html
    
### Metody
- reset_index(): Resetuje aktuální index a namísto něj použije výchozí hodnoty, viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html
- set_index(): Použije existující sloupec DataFrame jako index, viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html
- sort_index(): Seřadí hodnoty indexu, viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_index.html




In [48]:
# Import potrebnych knihoven
import pandas as pd

# Stazeni .csv souboru publikovaneho na github.com
url = 'https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv'
df = pd.read_csv(url,delimiter = ';')
df['DATE'] = pd.to_datetime(df['DATE'])
df.index.is_unique

True

## Multiindex

- DataFrame může mít víceúroňový index na řádcích i sloupcích
- Víceúrovňové indexy vznikají i při shlukování dat přes více řádků či při sumarizaci dat za pomoci kontingenční tabulky

## Vybrané metody pro práci s multiindexem

- droplevel(): Odstraní vybranou úroveň multiindexu, viz https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.droplevel.html
- sort_level(): Seřadí vybrou úroveň multiindexu, viz https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.sortlevel.html
- reorder_levels(): Změní pořadí úrovní multiindexu, viz https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.reorder_levels.html


In [49]:
srazky = df.copy()
# Nastaveni multiindexu
srazky = srazky.set_index(['DATE','NAME','STATION'])
# # Zmena poradi urovni multiindexu
srazky = srazky.reorder_levels(['NAME','STATION','DATE'])
# # Odstraneni multiindexu na urovni 1
srazky = srazky.droplevel(0,axis=0)



In [50]:
srazky1  = df.copy()
srazky1['ROK'] = srazky1['DATE'].dt.year
srazky1['MESIC'] = srazky1['DATE'].dt.month
# Agregace dat za pomoci kontingencni tabulky s nastavenim sloupcu jako indexu
srazky1 = srazky1.pivot_table(values = ['PRCP'],index=['ROK','MESIC'],columns='NAME',aggfunc='sum',fill_value='-')
srazky1.droplevel(0,axis=1)

NAME         GRAZ  KOSICE  MOSNOV PIDING PRAHA KLEMENTINUM  PRAHA LIBUS  \
ROK  MESIC                                                                
2000 1        9.2    18.9    30.7      -              17.0         29.4   
     2        0.5    36.6    20.8      -              18.3         18.5   
     3       67.6    31.6    31.2      -              81.4         48.5   
     4       18.1    43.4    40.4      -               6.5          2.9   
     5       90.9    57.3    71.7      -              38.7         10.9   
...           ...     ...     ...    ...               ...          ...   
2020 7       82.1    91.0    82.2  175.4                 -         21.5   
     8      227.0    62.8    86.9  257.9                 -        129.4   
     9       79.0    63.5   150.4  112.6                 -         65.3   
     10      38.9   107.6   162.9  155.2                 -         58.4   
     11      13.4     7.9    19.7      -                 -         10.5   

NAME        RUZYNE  SALZBURG  TURANY   WIEN  
ROK  MESIC                                   
2000 1        24.7      51.2    34.0   43.7  
     2        23.9      66.1    22.6   35.1  
     3        42.3     154.2    15.5   80.4  
     4         7.4      30.5     5.1   11.1  
     5        29.1     115.3    33.4   50.4  
...            ...       ...     ...    ...  
2020 7        34.5       0.0    55.3   27.1  
     8       138.0       0.0   104.5   33.5  
     9        58.9       0.0    75.8   91.2  
     10       70.3       8.6    95.0  120.9  
     11        7.3       0.0    20.4    7.1  

[251 rows x 10 columns]

Vytvořit dataframe, kde ve sloupcích budou NAME a na indexu ROK, MESIC
v hodnotách bunce suma PRCP

Agregovat výše uvedený dataframe jen podle ROK
ROK nasledne seradit sestupne

In [57]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv',delimiter = ';')
df['DATE'] = pd.to_datetime(df['DATE'])
df['ROK'] = df['DATE'].dt.year
df['MESIC'] = df['DATE'].dt.month

srazky1 = df.pivot_table(values = ['PRCP'],index=['ROK','MESIC'],columns='NAME',aggfunc='sum',fill_value='-')
srazky1 = srazky1.droplevel(1,axis=0).replace('-',0)

srazky1 = srazky1.groupby('ROK').sum()
srazky1.sort_index(ascending=False,inplace=True)
srazky1

PRCP                                                                \
NAME    GRAZ  KOSICE  MOSNOV  PIDING PRAHA KLEMENTINUM PRAHA LIBUS  RUZYNE   
ROK                                                                          
2020   630.2  1390.8  1007.9  1373.6               0.0       611.1   547.1   
2019   658.8  1374.9   653.1  1346.9               0.0       668.0   668.5   
2018   153.7   772.5   516.7  1212.7               0.0       454.9   444.4   
2017   780.9   553.0   750.6  1349.7               0.0      1231.5  1406.8   
2016  1120.6   735.7   643.2  1373.2               0.0       580.5   471.6   
2015   756.1   535.2   421.7   985.9               0.0       414.6   366.5   
2014   898.7   697.0   747.7  1180.0               0.0       665.4   625.0   
2013   647.8   616.3   581.3  1229.5               0.0       815.8   672.1   
2012   618.2   548.9   598.0  1255.7               0.0      1094.2   523.6   
2011   458.7   524.1   599.8  1103.1               0.0       775.3   454.6   
2010   914.4   958.9   896.7  1047.9               0.0       984.3   642.6   
2009  1204.3   613.1   710.5  1367.8               0.0       513.5   475.3   
2008   950.6   645.2   696.7  1139.8               0.0       494.2   486.0   
2007   826.4   630.9   708.8  1379.2               0.0       520.2   502.2   
2006   754.1   650.7   548.7    35.8               0.0       500.2   484.0   
2005   890.0   701.2   643.8     0.0              80.6       459.1   482.0   
2004   899.5   645.8   512.2     0.0             394.2       425.7   472.0   
2003   660.8   491.9   448.9     0.0             267.1       275.9   294.8   
2002   872.1   648.4   504.4     0.0             661.5       733.6   605.0   
2001   534.3   566.4   629.0     0.0             552.0       277.3   499.3   
2000   755.3   530.8   502.3     0.0             385.9       163.6   353.9   

                              
NAME SALZBURG  TURANY   WIEN  
ROK                           
2020      8.6   610.6  499.3  
2019    167.4   647.6  490.7  
2018    378.1   510.5  475.7  
2017   1605.7  1702.4  468.8  
2016   2026.5   616.5  599.5  
2015   1202.9   393.2  408.2  
2014   1311.6   602.0  665.7  
2013   1090.4   577.0  595.7  
2012    885.4   420.4  481.0  
2011    865.3   390.3  390.9  
2010    994.8   689.4  836.4  
2009   1401.3   557.8  899.6  
2008   1067.1   427.9  669.3  
2007   1169.5   479.4  863.3  
2006   1096.4   652.2  693.1  
2005   1214.4   574.9  595.4  
2004   1134.2   447.1  665.4  
2003    953.0   458.2  446.9  
2002   1387.2   481.6  813.5  
2001   1172.7   538.4  423.5  
2000   1165.7   404.9  537.5